# Solver Benchmark - Finding All Solutions

In [1]:
import time

def benchmark(name, function, param1, param2):
    """Benchmark a function with two parameters."""
    
    print('--'+ name + ' approach--')
    start_time = time.perf_counter()
    print('Number of models: ' + str(function(param1, param2)))
    end_time = time.perf_counter()
    print('Time: ' + str(round(end_time - start_time, 2)) + ' s')

## Z3

The `Z3` solver returns only one valid solution.
We try two ways to find/count all satisfying interpretations (models):
- Enumeration-based: Enumerate all possible assignments and check if they satisfy all assertions. This is possible in `Z3` with
  - conditional checking, which temporarily adds the literals corresponding to the assignment as further constraints and checks the resulting model.
  - substitution of the variables with their values, followed by simplification.
- [Solver-based](https://stackoverflow.com/questions/13395391/z3-finding-all-satisfying-models): Find the first model with the solver. Add the negation of its assignment as another constraint and re-run the solver. Thus, the solver will find a different solution. Repeat until unsatisfiable.

In [2]:
from z3 import *

def count_models_with_solver(solver, variables):
    solver.push() # as we will add further assertions to solver, checkpoint current state
    solutions = 0
    while solver.check() == sat:
        solutions = solutions + 1
        # Invert at least one variable to get a different solution:
        solver.add(Or([Not(x) if is_true(solver.model()[x]) else x for x in variables]))
    solver.pop() # restore solver to previous state
    return solutions

import itertools

# Fastest enumeration: conditional checking.
def count_models_by_enumeration(solver, variables):
    solutions = 0
    for assignment in itertools.product(*[(x, Not(x)) for x in variables]): # all combinations
        if solver.check(assignment) == sat: # conditional check (does not add assignment permanently)
            solutions = solutions + 1
    return solutions

# Creating the assignment as a separate step is slower.
def count_models_by_enumeration2(solver, variables):
    solutions = 0
    for assignment in itertools.product([False, True], repeat = len(variables)): # all combinations
        if solver.check([x if assign_true else Not(x) for x, assign_true in zip(variables, assignment)]) == sat:
            solutions = solutions + 1
    return solutions

# Using simplication instead of conditional checking is even slower.
def count_models_by_enumeration3(solver, variables):
    solutions = 0
    for assignment in itertools.product([BoolVal(False), BoolVal(True)], repeat = len(variables)): # all combinations
        satisfied = True
        for assertion in solver.assertions():
            if is_false(simplify(substitute(assertion, list(zip(variables, assignment))))):
                satisfied = False
                break
        if satisfied: solutions = solutions + 1
    return solutions

We try both approaches with a small propositional formula with 10 variables, using an AND constraint as well as an OR constraint.

In [3]:
from z3 import *

x = Bools(' '.join('x' + str(i) for i in range(10)))
solver = Solver()

print('## OR formula ##')
solver.add(Or(x))
benchmark('Solver-based', count_models_with_solver, solver, x)
benchmark('Enumeration-based (conditional check, direct assignment)', count_models_by_enumeration, solver, x)
benchmark('Enumeration-based (conditional check, separate assignment)', count_models_by_enumeration2, solver, x)
benchmark('Enumeration-based (substitute + simplify)', count_models_by_enumeration3, solver, x)

print('\n## AND formula ##')
solver.reset()
solver.add(And(x))
benchmark('Solver-based', count_models_with_solver, solver, x)
benchmark('Enumeration-based (conditional check, direct assignment)', count_models_by_enumeration, solver, x)
benchmark('Enumeration-based (conditional check, separate assignment)', count_models_by_enumeration2, solver, x)
benchmark('Enumeration-based (substitute + simplify)', count_models_by_enumeration3, solver, x)

## OR formula ##
--Solver-based approach--
Number of models: 1023
Time: 1.65 s
--Enumeration-based (conditional check, direct assignment) approach--
Number of models: 1023
Time: 0.08 s
--Enumeration-based (conditional check, separate assignment) approach--
Number of models: 1023
Time: 0.25 s
--Enumeration-based (substitute + simplify) approach--
Number of models: 1023
Time: 0.38 s

## AND formula ##
--Solver-based approach--
Number of models: 1
Time: 0.0 s
--Enumeration-based (conditional check, direct assignment) approach--
Number of models: 1
Time: 0.01 s
--Enumeration-based (conditional check, separate assignment) approach--
Number of models: 1
Time: 0.21 s
--Enumeration-based (substitute + simplify) approach--
Number of models: 1
Time: 0.39 s


The enumeration-based approach has to evaluate the same number of values for AND and OR formulas, i.e., all value combinations.
The conditional-checking approach still seems to benefit from problems with fewer solutions, though not as strong as the solver-based approach.
Overall, there is no clear winner: Depending on the number of solutions, solving or enumerating might be better.

## Google OR Tools

Google provides a framework for combinatorial white-box optimization problems, including constraint [solving](https://developers.google.com/optimization/cp/cp_solver) and [optimization](https://developers.google.com/optimization/cp/integer_opt_cp).
Besides the [Python API](https://developers.google.com/optimization/reference/python/sat/python/cp_model), C++, Java and C# are supported.
Creating an enumeration-based solution is more difficult than with `Z3`, as we cannot simply copy models or make conditional evaluations (temporary assignments).
Thus, we refrain from implementing such a solution.
Howeveer, as a nice alternative, iterating over all valid solutions is supported natively.

In [4]:
from ortools.sat.python import cp_model

def count_models_with_solver(model, variables):
    # TO DO: make a copy of model (not supported natively)
    solver = cp_model.CpSolver()
    solutions = 0
    while solver.Solve(model) == cp_model.FEASIBLE:
        solutions = solutions + 1
        # Invert at least one variable to get a different solution:
        model.AddBoolOr([x.Not() if solver.Value(x) == 1 else x for x in variables])
    return solutions

class Solution_Counter(cp_model.CpSolverSolutionCallback):

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1

    def solution_count(self):
        return self.__solution_count

def count_models_natively(model, counter_callback):
    solver = cp_model.CpSolver()
    solver.SearchForAllSolutions(model=model, callback=counter_callback)
    return counter_callback.solution_count()

In [5]:
from ortools.sat.python import cp_model

print('## OR formula ##')
model = cp_model.CpModel()
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolOr(x)
benchmark('Solver-based', count_models_with_solver, model, x)
model = cp_model.CpModel() # solver-based approach changes model, therefore re-creation
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolOr(x)
benchmark('Native', count_models_natively, model, Solution_Counter())

print('\n## AND formula ##')
model = cp_model.CpModel()
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolAnd(x)
benchmark('Solver-based', count_models_with_solver, model, x)
model = cp_model.CpModel()
x = [model.NewBoolVar('x' + str(i)) for i in range(10)]
model.AddBoolAnd(x)
benchmark('Native', count_models_natively, model, Solution_Counter())

## OR formula ##
--Solver-based approach--
Number of models: 1023
Time: 22.96 s
--Native approach--
Number of models: 1023
Time: 0.06 s

## AND formula ##
--Solver-based approach--
Number of models: 1
Time: 0.0 s
--Native approach--
Number of models: 1
Time: 0.0 s


The native approach beats the solver-based approach.
Compared to `Z3`, the native approach with `OR Tools` is about as fast as `Z3` (solver-based) for AND formulas and slightly faster than `Z3` (enumeration-based) for OR formulas.